# Финальное задание

In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

## Руководство по решению

Вам необходимо провести описанные ниже два этапа исследования, написать по результатам каждого этапа небольшой отчет (ниже указаны вопросы, ответы на которые должны содержаться в отчете), и предоставить для ревью данный отчет и код, с помощью которого вы выполнили задание.

**Обратите внимание:** высокое качество работы на кросс-валидации (близкое к 100%) — это в первую очередь повод задуматься о том, правильно ли вы обучаете модель. Возможно, вы заглядываете в будущее или настраиваетесь на неправильном наборе признаков.

### Подход 1: градиентный бустинг "в лоб"
Один из самых универсальных алгоритмов, изученных в нашем курсе, является градиентный бустинг. Он не очень требователен к данным, восстанавливает нелинейные зависимости, и хорошо работает на многих наборах данных, что и обуславливает его популярность. Вполне разумной мыслью будет попробовать именно его в первую очередь.

1\. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [2]:
features = pd.read_csv('./features.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
X = features.drop(["duration", "radiant_win", "tower_status_radiant", "tower_status_dire", "barracks_status_radiant", 
                   "barracks_status_dire"], axis=1)

2\. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [4]:
X.count()[X.count() / X.shape[0] < 1]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

3\. Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [5]:
X = X.fillna(0)

4\. Какой столбец содержит целевую переменную? Запишите его название.

In [6]:
y = features["radiant_win"]

5\. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [7]:
gen = KFold(shuffle=True, n_splits=5)

In [8]:
boosting_roc_auc = {}
boosting_timings = {}

In [ ]:
for n in np.arange(5, 50, 5):
    clf = GradientBoostingClassifier(n_estimators=n)
    begin = time.time()
    boosting_roc_auc[n] = roc_auc_score(y_true=y, y_score=cross_val_predict(estimator=clf, X=X, y=y, cv=gen, method="predict_proba", n_jobs=-1)[:,1])
    boosting_timings[n] = time.time() - begin

In [10]:
boosting_roc_auc

{5: 0.63416062558407371,
 10: 0.66400930664060565,
 15: 0.67593249989370308,
 20: 0.68111922232140709,
 25: 0.68655243571607139,
 30: 0.68963475968197274,
 35: 0.69259169902835416,
 40: 0.69467749553349667,
 45: 0.69573876461646711}

In [11]:
boosting_timings

{5: 9.703545808792114,
 10: 14.154901504516602,
 15: 18.61593461036682,
 20: 23.079106092453003,
 25: 31.387810945510864,
 30: 32.20297145843506,
 35: 37.44953179359436,
 40: 41.275766372680664,
 45: 46.75975561141968}

### Подход 2: логистическая регрессия

Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими для ускорения анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.

**Важно:** не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! Может пригодиться sklearn.preprocessing.StandartScaler.

In [12]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

1\. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [13]:
log_roc_auc = {}
log_timings = {}

In [14]:
for C in np.logspace(-5, 5, 11):
    clf = LogisticRegression(penalty="l2", C=C)
    begin = time.time()
    log_roc_auc[C] = roc_auc_score(y_true=y, y_score=cross_val_predict(estimator=clf, X=X_scaled, y=y, cv=gen, method="predict_proba", n_jobs=-1)[:,1])
    log_timings[C] = time.time() - begin

In [15]:
log_roc_auc

{1.0000000000000001e-05: 0.69509808495161096,
 0.0001: 0.71103179676041772,
 0.001: 0.71630593321610125,
 0.01: 0.71637988345087766,
 0.10000000000000001: 0.71637585833726014,
 1.0: 0.71657930283255999,
 10.0: 0.71621058378254854,
 100.0: 0.71652955454672618,
 1000.0: 0.71636876057901555,
 10000.0: 0.7163598006761025,
 100000.0: 0.71631043159834118}

In [16]:
log_timings

{1.0000000000000001e-05: 3.4579226970672607,
 0.0001: 4.2983245849609375,
 0.001: 6.114870071411133,
 0.01: 7.905963659286499,
 0.10000000000000001: 7.708338737487793,
 1.0: 8.280251502990723,
 10.0: 9.32875680923462,
 100.0: 8.27276611328125,
 1000.0: 8.466897010803223,
 10000.0: 8.673003435134888,
 100000.0: 8.3321533203125}

2\. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [17]:
categ_list = ["lobby_type"]
for i in range(1, 6):
    categ_list.append("r" + str(i) + "_hero")
    categ_list.append("d" + str(i) + "_hero")

In [18]:
X_log = scaler.fit_transform(X.drop(categ_list, axis=1))

In [19]:
log_roc_auc = {}
log_timings = {}

In [20]:
for C in np.logspace(-5, 5, 11):
    clf = LogisticRegression(penalty="l2", C=C)
    begin = time.time()
    log_roc_auc[C] = roc_auc_score(y_true=y, y_score=cross_val_predict(estimator=clf, X=X_log, y=y, cv=gen, method="predict_proba", n_jobs=-1)[:,1])
    log_timings[C] = time.time() - begin

In [21]:
log_roc_auc

{1.0000000000000001e-05: 0.69494333289128818,
 0.0001: 0.71133161476291418,
 0.001: 0.71626841703870359,
 0.01: 0.71645859136995027,
 0.10000000000000001: 0.71646228769533993,
 1.0: 0.71628590509813206,
 10.0: 0.71637621381834693,
 100.0: 0.71655901880210382,
 1000.0: 0.71652092978748405,
 10000.0: 0.71637447115599862,
 100000.0: 0.71634850874946832}

In [22]:
log_timings

{1.0000000000000001e-05: 3.233391761779785,
 0.0001: 4.001389741897583,
 0.001: 5.879321813583374,
 0.01: 7.201799392700195,
 0.10000000000000001: 7.620505094528198,
 1.0: 7.6075310707092285,
 10.0: 7.815138101577759,
 100.0: 7.818129062652588,
 1000.0: 7.88400411605835,
 10000.0: 7.7387824058532715,
 100000.0: 7.748761892318726}

Посмотрим на веса категориальных признаков в модели, где они присутствовали, чтобы понять, почему качество почти не поменялось

In [23]:
clf_with_categ = LogisticRegression(penalty="l2", C=0.1)
clf_with_categ.fit(X_scaled, y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
categ_indexes = []
for e in categ_list:
    categ_indexes.append(X.columns.get_loc(e))

In [48]:
other_indexes = list(range(X.columns.shape[0]))
for e in categ_indexes:
    other_indexes.remove(e)

In [56]:
np.mean(np.absolute(clf_with_categ.coef_[:,other_indexes])) / np.absolute(clf_with_categ.coef_[:,categ_indexes])

array([[  11.03317991,   10.37249878,  368.48329903,    2.85476529,
          21.29498819,   27.41352003,   16.5067511 ,   23.77391249,
           8.84489848,    4.98583172,   13.64432477]])

In [57]:
np.median(np.mean(np.absolute(clf_with_categ.coef_[:,other_indexes])) / np.absolute(clf_with_categ.coef_[:,categ_indexes]))

13.644324769936395

Получаем, что значения весов для категориальных признаков в 3-369 раз меньше, чем среднее значение весов, поэтому их удаление из выборки почти не вносит изменений в качество

3\. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [28]:
for i in range(1, 6):
    print(np.sort(X["r" + str(i) + "_hero"].unique())[-1])
    print(np.sort(X["d" + str(i) + "_hero"].unique())[-1])

112
112
112
112
112
112
112
112
112
112


В игре 112 различных идентификаторов героев

In [29]:
N = 112

4\. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [30]:
X_pick = np.zeros((X.shape[0], N))

for i, match_id in enumerate(X.index):
    for p in range(5):
        X_pick[i, X.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [31]:
X_pick.shape

(97230, 112)

In [32]:
X_log.shape

(97230, 91)

In [33]:
X_log_new = np.concatenate((X_log, X_pick), axis=1)

In [34]:
X_log_new.shape

(97230, 203)

5\. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [35]:
log_roc_auc = {}
log_timings = {}

In [36]:
for C in np.logspace(-5, 5, 11):
    clf = LogisticRegression(penalty="l2", C=C)
    begin = time.time()
    log_roc_auc[C] = roc_auc_score(y_true=y, y_score=cross_val_predict(estimator=clf, X=X_log_new, y=y, cv=gen, method="predict_proba", n_jobs=-1)[:,1])
    log_timings[C] = time.time() - begin

In [37]:
log_roc_auc

{1.0000000000000001e-05: 0.69921875391918964,
 0.0001: 0.72486809558618148,
 0.001: 0.74616527044316405,
 0.01: 0.75157941729831734,
 0.10000000000000001: 0.7518525492212248,
 1.0: 0.75200100769863409,
 10.0: 0.75205063480736301,
 100.0: 0.75192778198430876,
 1000.0: 0.75188778506583409,
 10000.0: 0.75169524439147617,
 100000.0: 0.75190618703790169}

In [38]:
log_timings

{1.0000000000000001e-05: 4.042310476303101,
 0.0001: 5.0329272747039795,
 0.001: 7.948398590087891,
 0.01: 11.79358172416687,
 0.10000000000000001: 15.230033159255981,
 1.0: 15.706127882003784,
 10.0: 15.393721342086792,
 100.0: 15.63925576210022,
 1000.0: 15.171144962310791,
 10000.0: 14.77789306640625,
 100000.0: 15.165686845779419}

6\. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [39]:
X_test = pd.read_csv('./features_test.csv', index_col='match_id')
X_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [40]:
X_test = X_test.fillna(0)

In [41]:
X_test_scaled = scaler.fit_transform(X_test.drop(categ_list, axis=1))

In [42]:
X_test_pick = np.zeros((X_test.shape[0], N))

for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_test_pick[i, X_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_test_pick[i, X_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [43]:
X_test = np.concatenate((X_test_scaled, X_test_pick), axis=1)

In [44]:
clf_final = LogisticRegression(penalty="l2", C=0.1)
clf_final.fit(X_log_new, y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [45]:
y_test = clf_final.predict_proba(X_test)

In [46]:
print(np.min(y_test))
print(np.max(y_test))
print(np.unique(y_test).shape[0])

0.00354133815476
0.996458661845
34354


## Проверка финальной модели

После того как вы провели все эксперименты и выбрали лучшую модель, можете проверить ее качество на тестовых матчах. Выборка тестовых матчей собрана в файле `matches_test.jsonlines.bz2`. В отличие от основного набора матчей, в тестовых матчах есть только та информация, которая известна на момент первых 5 игровых минут, результат матча — неизвестен. Таблица признаков для тестовых матчей — `features_test.csv`.

Для всех матчей из тестового набора предскажите вероятность победы Radiant, запишите предсказания в CSV файл с колонками `match_id` (идентификатор матча) и `radiant_win` — предсказанная вероятность. Файл с предсказаниями должен выглядеть примерно следующим образом:

```
match_id,radiant_win
1,0.51997370502
4,0.51997370502
15,0.51997370502
...
```

Отправьте решение на Kaggle в соревнование: Dota 2: Win Probability Prediction.

Ссылка на соревнование: [Dota 2: Win Probability Prediction](https://kaggle.com/join/coursera_ml_dota2_contest)

In [47]:
ans = {}
ans["match_id"] = pd.read_csv('./features_test.csv')["match_id"]
ans["radiant_win"] = y_test[:,1]
ans = pd.DataFrame(ans)
ans.to_csv("ans.csv", index=False)

В соревновании https://www.kaggle.com/c/dota-2-win-probability-prediction/ оценка получилась 0.75526 (примерно как на кросс-валидации).
![](https://pp.userapi.com/c845323/v845323265/121336/Pihla_iaA5s.jpg)